In [9]:
import os
import boto3

In [10]:
# Configura tu cliente de S3
s3 = boto3.client('s3')
bucket_name = 'datainternetaccess-cleaning'
s3_folder = '' 

In [11]:
# Define la ruta de tu carpeta local
local_folder = 'Limpieza/data'

In [12]:
# Subir los archivos al bucket de S3
def subir_a_s3(archivos, folder_local, bucket_name, s3_folder):
    for archivo in archivos:
        archivo_local = os.path.join(folder_local, archivo)
        s3_key = os.path.join(s3_folder, archivo).replace("\\", "/")  # Ruta del archivo en S3 asegurando formato correcto
        try:
            print(f"Subiendo {archivo_local} a S3...")
            s3.upload_file(archivo_local, bucket_name, s3_key)
            print(f"{archivo_local} subido con éxito a S3!")
        except Exception as e:
            print(f"Error subiendo {archivo}: {str(e)}")


In [13]:
# Función para obtener archivos de S3
def get_s3_files(bucket_name, s3_folder):
    s3_files = []
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder)
    if 'Contents' in response:
        for obj in response['Contents']:
            # Obtenemos los nombres de los archivos en S3 (sin el prefijo, ya que estamos en la raíz)
            s3_files.append(obj['Key'].replace(s3_folder, '').lstrip('/'))
    return s3_files

In [14]:
# Función para obtener archivos locales
def get_local_files(local_folder):
    local_files = []
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            # Obtenemos la ruta relativa para comparar con los nombres de S3
            relative_path = os.path.relpath(os.path.join(root, file), local_folder)
            local_files.append(relative_path.replace("\\", "/"))  # Asegurar formato de ruta
    return local_files

In [15]:
# Compara los archivos de S3 con los archivos locales
def sincronizar_archivos(bucket_name, local_folder, s3_folder):
    s3_files = get_s3_files(bucket_name, s3_folder)
    local_files = get_local_files(local_folder)

    # Archivos que están en S3 pero no en la carpeta local
    new_files = set(s3_files) - set(local_files)

    # Descargar archivos nuevos desde S3
    for file in new_files:
        # Descargar el archivo desde S3
        local_path = os.path.join(local_folder, file)
        s3_key = f"{s3_folder}{file}".replace("\\", "/")

        # Crear los directorios locales si no existen
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        try:
            print(f"Descargando {file} desde {s3_key} a {local_path}")
            s3.download_file(bucket_name, s3_key, local_path)
        except Exception as e:
            print(f"Error descargando {file}: {str(e)}")

    # Archivos que están localmente pero no en S3
    archivos_para_subir = set(local_files) - set(s3_files)
    if archivos_para_subir:
        subir_a_s3(archivos_para_subir, local_folder, bucket_name, s3_folder)


In [16]:
# Llamamos la función para sincronizar archivos
sincronizar_archivos(bucket_name, local_folder, s3_folder)

Subiendo Limpieza/data/progreso_coordenadas.csv a S3...
Limpieza/data/progreso_coordenadas.csv subido con éxito a S3!
Subiendo Limpieza/data/municipios_sin_coordenadas_completas.csv a S3...
Limpieza/data/municipios_sin_coordenadas_completas.csv subido con éxito a S3!
Subiendo Limpieza/data/coordenadas_unicas.csv a S3...
Limpieza/data/coordenadas_unicas.csv subido con éxito a S3!
